<a href="https://colab.research.google.com/github/mukundanr/GenAI-Labs/blob/main/Lab2-Gen-Response-with-RAG/FIRE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio


In [ ]:
!pip install transformers faiss-cpu sentence-transformers langchain==0.0.354 pypdf openai==1.3.9 python-dotenv==1.0.0 PyMuPDF==1.24.2 tqdm tiktoken

In [ ]:
import gradio as gr
import faiss
import numpy as np
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
import fitz
import tiktoken
import os
from dotenv import load_dotenv
from google.colab import userdata

# Load environment variables
load_dotenv()
os.environ["OPENAI_API_KEY"] = userdata.get('open_ai_key')
model_name = "gpt-3.5-turbo"
token_encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
encoder = SentenceTransformer("all-MiniLM-L6-v2")
knowledge_base = []
embed_index = None
client = OpenAI()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00


In [ ]:
import PyPDF2
from google.colab import files

def read_pdf(file):
    pdf_reader = PyPDF2.PdfReader(file)
    pdf_content = ''
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        pdf_content += page.extract_text() if page.extract_text() else ''
    return pdf_content

# Prompt the user to upload a PDF file
uploaded_prompt = files.upload()

# Process the uploaded file
for file_name in uploaded_prompt.keys():
    with open(file_name, 'rb') as file:
        content_prompt = read_pdf(file)

Saving FIRE 4U prompt_v2.pdf to FIRE 4U prompt_v2 (1).pdf


In [ ]:
# Prompt the user to upload a PDF file
uploaded_statement = files.upload()

# Process the uploaded file
for file_name in uploaded_statement.keys():
    with open(file_name, 'rb') as file:
        content_statement = read_pdf(file)

Saving Vanguard_Synthetic_Statement_Jamie_Preston.pdf to Vanguard_Synthetic_Statement_Jamie_Preston (2).pdf


In [ ]:
# Global variables for chunk size and overlap
chunk_size = 500
chunk_overlap = 50

In [ ]:
#This is needed to use the RecursiveCharacterTextSplitter
!pip install langchain

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

#Initialize the knowledge base
knowledge_base = []

# The 'global' keyword should not be indented
global knowledge_base

recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
)
for file_name in uploaded_prompt.keys():
    with open(file_name, 'rb') as file:
        # Assuming 'read_pdf' is defined elsewhere
        from PyPDF2 import PdfReader
        def read_pdf(file):
            pdf_reader = PdfReader(file)
            pdf_content = ''
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                pdf_content += page.extract_text() if page.extract_text() else ''
            return pdf_content

        content_prompt = read_pdf(file)
    chunks = recursive_splitter.split_text(content_prompt)
    knowledge_base.extend(chunks)

In [ ]:
# Check if knowledge_base is empty
if not knowledge_base:
    print("Error: knowledge_base is empty. Check the PDF file and splitting process.")
else:
    vectors = encoder.encode(knowledge_base)

    # Check the shape of vectors and print a message if it's not 2D
    if vectors.ndim != 2:
        print(f"Error: vectors has shape {vectors.shape}, expected 2 dimensions. Check the encoder output.")
    else:
        vector_dimension = vectors.shape[1]
        embed_index = faiss.IndexFlatL2(vector_dimension)
        faiss.normalize_L2(vectors)
        embed_index.add(vectors)

In [ ]:
retrieved_relevant_chunk = []
search_vector = encoder.encode("Provide asset Summary Table")
_vector = np.array([search_vector])
faiss.normalize_L2(_vector)

# Define the number of results to retrieve
results_len = 10  # For example, retrieve the top 10 results

distances, retrieved_idxs = embed_index.search(_vector, results_len)

for i in range(len(retrieved_idxs.ravel())):
    retrieved_relevant_chunk.append(knowledge_base[retrieved_idxs.ravel()[i]])

#return "\n\n".join(answer)
print(retrieved_relevant_chunk)

['format" tab  in the  "Format Sheet" . \n○ Structure the table as follows:  \n■ Column 1:  Account name (e.g., "Bain & Co 401K").  \n■ Column 2:  Account value.  \n■ Column 3:  Asset class breakdown (stocks, bonds, cash).  \n○ In the table please aslo Include:  \n■ A sum of all assets.  \n■ The overall asset class breakdown  for total assets.To calculate this \ncombine all the assets first and then break them down by stocks, bonds \nand cash.  \n○ Keep the table concise (no more than half a page).', 'Step 1: Request documents  \n1. Ask clients documents to upload with below prompt”  \n○ “Please upload your latest tax returns  and financial statements  for us to \nunderstand your financial situation and assets to project your path to FIRE”  \nKeep this query very succint.  \n■ Instruction : These are essential for projecting retirement age and asset \ndrawdown.  \nStep 2: Provide asset Summary Table:  \n○ Provide a succinct summary of the client’s assets using the "asset class', 'Instr

In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "\n\n".join(retrieved_relevant_chunk)}, # Join the list of strings into a single string
    {"role": "user", "content": content_statement}
  ]
)

summary = response.choices[0].message.content
print(summary)

**Asset Summary Table:**

| Account Name       | Account Value | Asset Class Breakdown       |
|--------------------|---------------|-----------------------------|
| Individual Brokerage | $550,000     | Stocks: 70%, Bonds: 20%, Cash: 10% |
| Roth IRA           | $160,000     | Stocks: 95%, Bonds: 5%           |
| Traditional IRA    | $290,000     | Stocks: 50%, Bonds: 50%          |
| **Total**            | **$1,000,000** |                               |

**Overall Asset Class Breakdown**:  
- Stocks: 63.75%  
- Bonds: 26.25%  
- Cash: 10.00%  

---
Client Confirmation:  
"Are these all the assets you want me to consider for your 'FIRE4U - Financial Independence' calculations? If any are missing, please upload the documents or enter the information."

---

**Fire Calculation Assumptions:**

- **Current Age:** 39  
- **Portfolio Rate of Return:** 7% based on 80% stock, 20% bond portfolio  
- **Estimated Future Annual Savings:** $180,000 assuming 30% saving rate post-tax; Year-over-yea

In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Provide an asset summary table"}, # Join the list of strings into a single string
    {"role": "user", "content": content_statement}
  ]
)

summary = response.choices[0].message.content
print(summary)

| Account Type | Total Value | Asset Allocation |
| --- | --- | --- |
| Individual Brokerage | $550,000 | 70% Stocks, 20% Bonds, 10% Cash |
| Roth IRA | $160,000 | 95% Stocks, 5% Bonds |
| Traditional IRA | $290,000 | 50% Bonds, 50% Stocks |
| **Total Portfolio** | **$1,000,000** | **100%** |

| Asset Allocation | Allocation (%) | Current Value |
| --- | --- | --- |
| Domestic Stocks | 50% | $500,000 |
| International Stocks | 20% | $200,000 |
| Bonds | 25% | $250,000 |
| Cash & Equivalents | 5% | $50,000 |
| **Total** | **100%** | **$1,000,000** |

Year-to-Date Performance:
- Starting Balance: $970,000
- Contributions: $20,000
- Withdrawals: $0
- Investment Growth: $10,000
- **Ending Balance: $1,000,000**

Fees & Expenses:
- Fund Expense Ratios: $950
- Advisory Fees: $650
- Transaction Costs: $0
- **Total Fees: $1,600**


In [ ]:
# prompt: Step 3: Ask Additional Questions for Retirement Age Estimation
# Ask the client the following questions to refine the estimate:
# 1. Age of family members:
# 2. Monthly Expenditures: "What are your approximate monthly expenditures, including mortgage payments?"
# 3. Retirement Location: "Can I assume you will retire in your current city of residence, XYZ? If not, please provide your desired retirement location."
# 4. Upcoming Expenses: "Are there large, upcoming expenses (e.g. home renovations) before retirement? For college expenses, only provide amounts not covered by college investment funds."
# Avoid asking, "When would you like to retire?"
# Ask each question separately and store input from user

client_data = {}

# 1. Age of family members
family_ages = input("Please provide the ages of your family members (e.g., 60, 65, 70, or 'N/A' if not applicable): ")
client_data["family_ages"] = family_ages

# 2. Monthly Expenditures
monthly_expenditures = input("What are your approximate monthly expenditures, including mortgage payments?: ")
client_data["monthly_expenditures"] = monthly_expenditures

# 3. Retirement Location
retirement_location = input("Can I assume you will retire in your current city of residence, California? If not, please provide your desired retirement location: ")
client_data["retirement_location"] = retirement_location

# 4. Upcoming Expenses
upcoming_expenses = input("Are there large, upcoming expenses (e.g. home renovations) before retirement? For college expenses, only provide amounts not covered by college investment funds: ")
client_data["upcoming_expenses"] = upcoming_expenses

client_data

Please provide the ages of your family members (e.g., 60, 65, 70, or 'N/A' if not applicable): 30,25,6
What are your approximate monthly expenditures, including mortgage payments?: 5000
Can I assume you will retire in your current city of residence, California? If not, please provide your desired retirement location: NA
Are there large, upcoming expenses (e.g. home renovations) before retirement? For college expenses, only provide amounts not covered by college investment funds: 800


{'family_ages': '30,25,6',
 'monthly_expenditures': '5000',
 'retirement_location': 'NA',
 'upcoming_expenses': '800'}

In [ ]:
# prompt: Step 4: Make key Assumptions for Retirement Calculation
# Please present these assumptions to the client in a table format using the ‘Fire calculation table’ provided below
# 1. Family Members: List ages of all family members. Here is the format “Member 1, salaried, age 29, Member 2, working, age 39, member 3, not working, age 6”
# ○ Instruction: Current age is critical. If you do not have a good estimate for this information, please ask the user. Do not move forward without this.
# 2. Asset Portfolio Growth Rate (Nominal): Clearly state assumptions for growth rates by asset type based on historical data or client inputs.
# 3. Projected Future Annual Savings: Project based on 35% post-tax savings rate and provide $ value. Assume salary will grow 5% every year till retirement. Assume appropriate split in retirement vs. non-retirement accounts.
# 4. Estimated Monthly Expenditure in Retirement:

def create_fire_calculation_table(client_data):
    """Creates a FIRE calculation table based on client data."""

    # Sample data (replace with actual client inputs from client_data)
    family_members = client_data.get("family_ages", "N/A")

    # Split the family_members string into a list of members and ages
    family_details = []
    if family_members != "N/A":
      for member_info in family_members.split(','):
        member_info = member_info.strip()
        parts = member_info.split("age")
        if len(parts) == 2:
            name_status = parts[0].strip()
            age = parts[1].strip()
            family_details.append(f"{name_status}, age {age}")

    asset_growth_rates = {
        "Stocks": 0.07,
        "Bonds": 0.04,
        "Real Estate": 0.05
    }
    projected_annual_savings = 50000 # Placeholder - should come from client input
    monthly_expenditure_before = int(client_data.get("monthly_expenditures", 0)) # Use client input
    monthly_expenditure_after = monthly_expenditure_before * 0.8 # Example: 80% of pre-retirement expenditure

    # Create a pandas DataFrame for the table
    data = {
        "Assumption": [
            "Family Members",
            "Asset Portfolio Growth Rate (Nominal)",
            "Projected Future Annual Savings",
            "Estimated Monthly Expenditure in Retirement (Before Age)",
            "Estimated Monthly Expenditure in Retirement (After Age)"
        ],
        "Details": [
            ", ".join(family_details) if family_details else "N/A",
            ", ".join([f"{asset}: {rate * 100:.1f}%" for asset, rate in asset_growth_rates.items()]),
            f"${projected_annual_savings:,}",
            f"${monthly_expenditure_before:,}",
            f"${monthly_expenditure_after:,}"
        ]
    }

    # 6. Post-Retirement Tax Rate
    retirement_location = client_data.get("retirement_location", "California")
    post_retirement_tax_rate = 0.20  # Default assumption

    # Placeholder for more sophisticated tax rate calculation based on location
    if retirement_location == "California":
      # In reality, you would implement a more precise tax calculation
      # potentially using external tax rate data APIs or other sources.
      post_retirement_tax_rate = 0.12 # Example, replace with actual calculation
    elif retirement_location == "Nevada": #another example
      post_retirement_tax_rate = 0.05 # Example, replace with actual calculation

    # ... (Rest of your existing code)

    # Add post-retirement tax rate to the table
    data["Assumption"].append("Post-Retirement Tax Rate")
    data["Details"].append(f"{post_retirement_tax_rate*100:.1f}%")

    # 7. Social Security Income Estimate (Placeholder)
    # Replace with an actual estimate based on income data
    husband_ss_income = 25000  # Replace with actual estimate
    wife_ss_income = 20000  # Replace with actual estimate
    ss_start_age = 67 # Replace with actual age

    data["Assumption"].extend(["Husband's Social Security Income", "Wife's Social Security Income", "Social Security Start Age"])
    data["Details"].extend([f"${husband_ss_income:,}", f"${wife_ss_income:,}", f"{ss_start_age}"])


    # 8. Child’s Education
    data["Assumption"].append("Child's Education")
    data["Details"].append("Assuming a 529 Scholar account is set up separately for expenses.")

    # 9. Home Mortgage
    data["Assumption"].append("Home Mortgage")
    data["Details"].append("Assuming your house mortgage is paid off at retirement or income covers payments post-retirement.")

    df = pd.DataFrame(data)
    return df

# Example usage (assuming client_data is populated from user input)
fire_table = create_fire_calculation_table(client_data)
fire_table

,Assumption,Details
0,Family Members,N/A
1,Asset Portfolio Growth Rate (Nominal),"Stocks: 7.0%, Bonds: 4.0%, Real Estate: 5.0%"
2,Projected Future Annual Savings,"$50,000"
3,Estimated Monthly Expenditure in Retirement (B...,"$5,000"
4,Estimated Monthly Expenditure in Retirement (A...,"$4,000.0"
5,Post-Retirement Tax Rate,20.0%
6,Husband's Social Security Income,"$25,000"
7,Wife's Social Security Income,"$20,000"
8,Social Security Start Age,67
9,Child's Education,Assuming a 529 Scholar account is set up separ...
